Before we also want to learn about the convolutional neural network in details:

What is a convolutional neural network ? Why are CNNs very popular ? What is the twchnique for CNN ? 
Convolutional neural network (CNN) is a class of deep feed forward artificial neural network. 
CNN has been successfully applied for analyzing the visual imagery. 

A CNN consists of an input and output layer and a number of hidden layers. The hidden layers are either convolutional, pooling or fully connected. 

A convolutional layer apply the convolution operation to the input, passing the result to the next layer. Three hyperparameters control the size of the output volume of the convolutional layer - the depth of the output volume, stride and zero padding. 

The depth of the output volume controls the number of neurons in a layer that connect to the same region of the input volume. 

Stride controls how the depth columns around the spatial dimensions (width and height ) are allocated. When the stride is 1, we move the filters one pixel at a time. 

Zero padding helps to preserve the size of the input volume. 


Pooling is one of the important features of CNN, which is a form of non-linear down-sampling. Out of the many pooling functions, max-pooling is the most common. It partitions the input image into a set of non-overlapping rectangles and, for each such sub-region, outputs the maximum. The most common form is a pooling layer with filters of size 2x2 applied with a stride of 2 downsamples at every depth slice in the input by 2 along both width and height, discarding 75% of the activations. In this case, every max operation is over 4 numbers. The depth dimension remains unchanged. 


After several convolutional and max pooling layers, the high-level reasoning in the neural network is done via fully connected layers. Neurons in a fully connected layer have connections to all activations in the previous layer, as seen in regular neural networks. Their activations can hence be computed with a matrix multiplication followed by a bias offset.

Here, we want to build a multi layer convolutional neural network and visualize all the steps : 

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


The first step in creating the multi-lyaer convolutional network is to define all the weights and biases. In each layer we need to initialize the weights and biases. We initialize the weights with small amount of noisy for symmetry breaking and also initialized slightly positive biases to avoid the dead neurons. 

Let us define a function which achieves that:

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.15)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

As can be observed, we start with the MNIST 28×28 greyscale images of digits.  We then create 32, 5×5 convolutional filters / channels plus ReLU (rectified linear unit) node activations.  After this, we still have a height and width of 28 nodes.  We then perform down-sampling by applying a 2×2 max pooling operation with a stride of 2.  Layer 2 consists of the same structure, but now with 64 filters / channels and another stride-2 max pooling down-sample.  We then flatten the output to get a fully connected layer with 3164 nodes, followed by another hidden layer of 1000 nodes.  These layers will use ReLU node activations.  Finally, we use a softmax classification layer to output the 10 digit probabilities.

Similarly, we also define the functions for convolution and pooling. Convolutions use the stride of 1 and zero padded so that the size of the input and the output remains the same. 

The pooling is the plain old max pooling over 2 x 2 blocks.

In [3]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

TensorFlow has a handy loader for the MNIST data which is sorted out in the first couple of lines.  Next, we declare a placeholder for the image input data, x.  The image input data will be extracted using the mnist.train.nextbatch() function, which supplies a flattened 28×28=784 node, single channel greyscale representation of the image. However, before we can use this data in the TensorFlow convolution and pooling functions, such as conv2d() and max_pool() we need to reshape the data as these functions take 4D data only.

The format of the data to be supplied is [i, j, k, l] where i is the number of training samples, j is the height of the image, k is the weight and l is the channel number.  Because we have a greyscale image, l will always be equal to 1 (if we had an RGB image, it would be equal to 3).  The MNIST images are 28 x 28, so both j and k are equal to 28.  When we reshape the input data x into x_shaped, theoretically we don’t know the size of the first dimension of x, so we don’t know what i is.  However, tf.reshape() allows us to put -1 in place of i and it will dynamically reshape based on the number of training samples as the training is performed.  So we use [-1, 28, 28, 1] for the second argument in tf.reshape().


First convolutional layer is implemented as : 

In [4]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [5]:
x = tf.placeholder(tf.float32,shape=[None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])
print(x_image)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)


We then convolve x_image with the weight tensor, add the bias and apply the ReLU (Rectifier Linear Unit) activation function and finally the max pool. The max_pool reduces the image dimension to 14x14

In [6]:
conv1 = conv2d(x_image,W_conv1)+b_conv1
h_conv1 = tf.nn.relu(conv1)

In [7]:
h_pool1 = max_pool_2x2(h_conv1)
print(h_pool1)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


We now construct the second convolution layer. The second convolution layer will have 64 features for each 5x5 patch. The weights and biases for the second layer is then defined as:

In [8]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

In [9]:
conv2 = conv2d(h_pool1, W_conv2)+b_conv2
h_conv2 = max_pool_2x2(conv2)
h_pool2 = tf.nn.relu(h_conv2)
print(h_pool2)

Tensor("Relu_1:0", shape=(?, 7, 7, 64), dtype=float32)


Next layer is called the densely connected layer. We add a fully connected layer with 1024 neurons to allow processing of the entire image. The tensor from the pooling layer is first reshaped to a batch of vectors, which is multiplied by a weight matrix. We then add bias and apply Relu.

In [10]:
#Define weight and bias
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

#Reshape h_pool2
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

After the densely connected layer, we use the dropout to reduce the overfitting. Then only we apply the readout layer as the final layer. We create a placeholder for the probability that the neurons output is kept during the dropout. This allows us to turn the dropout during the training and turn it off during the testing. 

In [11]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Finally we add the readout layer. The final layer to read the output, like for the softmax regression. 

In [15]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

y_ = tf.placeholder(tf.float32,shape= [None,10])

The final step would be the training and evaluating the model, which is achieved in a similar way as observed in the previous cases:

In [21]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction =tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch = mnist.train.next_batch(50)
        if i% 100 ==0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
            print('step %d,  training accuracy %g' %(i, train_accuracy))
        train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    print('Test accuracy %g' %(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})))
                               

step 0,  training accuracy 0.16
step 100,  training accuracy 0.72
step 200,  training accuracy 0.9
step 300,  training accuracy 0.9
step 400,  training accuracy 0.92
step 500,  training accuracy 0.92
step 600,  training accuracy 0.88
step 700,  training accuracy 0.88
step 800,  training accuracy 0.9
step 900,  training accuracy 0.96
Test accuracy 0.9482
